# Part 2 - Gauge Freedom and Symmetries

In [1]:
#widget has bugs with reloading plot
#%matplotlib widget
%matplotlib inline
#TODO: widget works, but size is not displayed correctly

import numpy as np 
from scipy.sparse import diags # Used for banded matrices

import matplotlib as mpl
import matplotlib.pyplot as plt # Plotting
from cycler import cycler


import seaborn as sns
plt.style.use('seaborn-dark')
plt.rcParams.update({'font.size':14})
#mpl.rcParams['figure.dpi']= 100

from IPython.display import display, Markdown, Latex, clear_output # used for printing Latex and Markdown output in code cells
from ipywidgets import Layout, fixed, HBox, VBox #interact, interactive, interact_manual, FloatSlider, , Label, Layout, Button, VBox
import ipywidgets as widgets

import functools
import time, math
from scipy.linalg import expm 

# Gauge Freedom
<!---  Define a few convenience macros for bra-ket notation. -->
$\newcommand{\ket}[1]{\left\vert{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right\vert}$
$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$
$\newcommand{\dyad}[2]{\left|{#1}\middle\rangle\middle\langle{#2}\right|}$
$\newcommand{\mela}[3]{\left\langle #1 \vphantom{#2#3} \right| #2 \left| #3 \vphantom{#1#2} \right\rangle}$
$\newcommand\dif{\mathop{}\!\mathrm{d}}$
$\newcommand\ii{\mathrm{i}}$
$\newcommand{\coloneqq}{\mathop{:=}}$


A wavefunction $\ket{\psi}$ is not observable and thus not necessarily unique. One can add an arbitrary phase to each orbital of our $n$-chain, which would change the wavefunction, but not the underlying physics, as probabilities of an outcome $O_i$ are calculated using the norm of the projection of the probed wavefunction onto the $i$-th eigenstate of the operator $|\braket{\phi_i}{\psi}|^2$. The same holds for Operators, whose representation depends on the chosen picture. Observables however are unique and correspond to the eigenvalues of a hermitian operator $\hat O $.

In the following the phase $\exp(\ii \theta_{ij})$ accumulated for hopping from one orbital $i$ to another $j$ can be changed. Observe how on the one hand the matrix elements of the Hamiltonian $H_{ij}$ change, but on the other hand the eigenvalues (and thus the possible observables) stay the same. The values of $\theta_{ij}$ are calculated using a `phase` vector as input
\begin{equation}
    \theta_{ij} = \mathrm{phase}[i] - \mathrm{phase}[j].
\end{equation}
This guarantees that no matter which path we take from any given initial position back to the same position, we will always end up with a net phase of zero and thus preserve the structure of the Hamiltonian.


In [2]:
#TODO: possibly implement triu or tril for hamilton matrix
def Hopping_Matrix_with_Phase(n = 6, phase=None):
    """
    TODO: write documentation
    """

    ### Check if system is large enough, i.e. if n=>2
    assert n >= 2, "error n must be greater or equal to 2"

    diagonal_entries = [np.ones(n-1), np.ones(n-1)]
    H = diags(diagonal_entries, [-1, 1]).toarray()
    #TODO: check length of phase vector
    # take care of the values not on the lower and upper main diagonal
    H[[0, n-1], [n-1, 0]] = 1
    
    if phase is not None:
        H = H.astype(complex) #otherwise float conversion error
        #Calculte H * exp(1*theta_ij)
        H *= np.exp(1j*(phase[None, :] - phase[:, None]))

    return H


phase_dict = {
    2:[[1,2], [0,2], [-1, 0.45], [np.pi, -np.pi],
    3:[[1,2-,3], [0,2,4], [-1,5,-0.45], [np.pi, -np.pi, -np.pi]],
    4:[[1,2,3,4], [0,2,4,8], [-1,5,-2,0.45], [np.pi, -np.pi, -np.pi,np.pi]-],
    5:[[1,2,3,4,5], [0,2,4,6,8], [-1,5,-2,1.5,0.45], [np.pi, -np.pi, -np.pi,np.pi,-np.pi]],
    6:[[1,2,3,4,5,6], [0,2,4,6,8,10], [-1,5,-2,1.5,-0.83,0.45], [np.pi, -np.pi, -np.pi,np.pi,-np.pi,np.pi]],
    7:[[1,2,3,4,5,6,7], [0,2,4,6,8,10,12], [-1,5,-2,1.5,-0.83,0.7,0.45], [np.pi, -np.pi, -np.pi,np.pi,-np.pi,np.pi,-np.pi]],
    8:[[1,2,3,4,5,6,7,8], [0,2,4,6,8,10,12,14], [-1,5,-2,1.5,-0.83,0.7,-2,0.45], [np.pi, -np.pi, -np.pi,np.pi,-np.pi,np.pi,-np.pi,np.pi]],
    9:[[1,2,3,4,5,6,7,8,9], [0,2,4,6,8,10,12,14,16], [-1,5,-2,1.5,-0.83,0.7,4,-1,0.45], [np.pi, -np.pi, -np.pi,np.pi,-np.pi,np.pi,-np.pi,np.pi,-np.pi]],
    10:[[1,2,3,4,5,6,7,8,9,10], [0,2,4,6,8,10,12,14,16,18], [-1,5,-2,1.5,-0.83,0.7,1,-3,-10,0.45], [np.pi, -np.pi, -np.pi, np.pi, -np.pi, np.pi, -np.pi, np.pi, -np.pi, np.pi]]
    }

In [3]:
H = Hopping_Matrix_with_Phase()

In [4]:
rng = np.random.default_rng(seed=5)

array([[-0.4+0.j , -0.6+0.j , -0.1+0.j ,  0.6+0.j , -0.1+0.j ,  0.4+0.j ],
       [-0.4-0.1j, -0.3-0.1j,  0.5+0.1j, -0.2-0.j ,  0.5+0.1j, -0.4-0.1j],
       [-0.4-0.2j,  0.2+0.1j,  0.5+0.2j, -0.3-0.2j, -0.4-0.2j,  0.4+0.2j],
       [-0.4-0.1j,  0.6+0.1j,  0.1+0.j ,  0.6+0.1j, -0.1-0.j , -0.4-0.1j],
       [ 0.3+0.3j, -0.2-0.3j,  0.3+0.4j,  0.1+0.1j, -0.3-0.4j, -0.3-0.3j],
       [-0.4+0.1j, -0.2+0.j , -0.5+0.j , -0.4+0.j , -0.4+0.j , -0.4+0.1j]])

TODO: add theory to magnetic flux and matrix
TODO: add slider and widgets to play around with the phase
TODO: add default phases

# Symmetry

### Abelian group of translations
TODO add theory about group of translations, definition, properties, qm definiton via commutator
Plot matrices

In [15]:
def Right_Translation_Matrix(n=6):
    """
    TODO: write documentation
    """

    ### Check if system is large enough, i.e. if n=>2
    assert n >= 2, "error n must be greater or equal to 2"

    diagonal_entries = [np.ones(n-1)]
    P = diags(diagonal_entries, [1]).toarray()
    # Translation from position n to 1
    P[n-1, 0] = 1

    return P

def Translation_Group(n=6):
    """
    TODO: write documentation
    """
    P = Right_Translation_Matrix(n=n)
    return np.array([np.linalg.matrix_power(P, i) for i in np.arange(n)])

In [16]:
P = Right_Translation()

In [17]:
Translation_Group(n=4)

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]],

       [[0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]])

TODO calculate commutator and write about it

In [18]:
print(P @ H - H @ P)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


reflection symmetry theory

In [104]:
def Reflection_Matrix(n=6, pos=0):
    """
    TODO: write documentation
    """

    ### Check if system is large enough, i.e. if n=>2
    assert n >= 2, "error n must be greater or equal to 2"

    return np.roll(np.fliplr(np.eye(n)), shift=pos+1, axis=0)

def All_Reflections(n=6):
    return np.array([Reflection_Matrix(n, pos=i) for i in np.arange(n)])

In [118]:
Reflection_Matrix()

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]])

In [116]:
All_Reflections(n=6)

array([[[1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0.]],

       [[0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0.]],

       [[0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
      

In [20]:
R = Reflection_Matrix()
for i in np.arange(2, 6):
    print(Reflection_Matrix(n=i))
    print()

[[1. 0.]
 [0. 1.]]

[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]

[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]

[[1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]]



In [21]:
print(R @ H - H @ R)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [119]:
print(R @ P - P @ R)

[[ 0.  1.  0.  0.  0. -1.]
 [ 1.  0.  0.  0. -1.  0.]
 [ 0.  0.  0. -1.  0.  1.]
 [ 0.  0. -1.  0.  1.  0.]
 [ 0. -1.  0.  1.  0.  0.]
 [-1.  0.  1.  0.  0.  0.]]


In [23]:
np.linalg.matrix_power(R,4)

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [25]:
R

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]])

In [124]:
np.linalg.eigvals(P)

array([-1. +0.j       , -0.5+0.8660254j, -0.5-0.8660254j,  0.5+0.8660254j,
        0.5-0.8660254j,  1. +0.j       ])

In [125]:
_, pvec = np.linalg.eig(P)

In [130]:
np.round(pvec.T.conj() @ H @ pvec,0).real

array([[-2.,  0.,  0., -0., -0., -0.],
       [ 0., -1.,  0.,  0., -0.,  0.],
       [ 0.,  0., -1., -0.,  0.,  0.],
       [-0.,  0., -0.,  1.,  0., -0.],
       [-0., -0.,  0.,  0.,  1., -0.],
       [-0.,  0.,  0., -0., -0.,  2.]])

In [138]:
import test_import 
